Este notebook pretende analisar a preservação estrutural de series temporais após redução de dimensionalidade. Para isso serão selecionados datasets com alta dimensionalidade da biblioteca AEON e aplicados variações do algoritimo do PAA (Piecewise Aggregate Approximation) para redução de dimensionalidade aplicando diferentes taxas perceptuais de redução.

Para medir essa preservação da estrutura iremos armazenar uma matriz de similaridade dos K vizinhos mais próximos de cada série temporal antes e depois da redução de dimensionalidade utilizando a distância euclidiana. Assim poderemos comparar se após a redução de dimensionalidade os K vizinhos mais próximos de cada série temporal permanecem os mesmos.

A métrica sera normalizada para representar valores entre 0 e 1, onde 1 representa que todos os K vizinhos mais próximos permaneceram os mesmos após a redução de dimensionalidade e 0 representa que nenhum vizinho próximo permaneceu o mesmo.



In [1]:
# Importa algumas bibliotecas necessárias
import numpy as np
import pandas as pd
import os

from aeon.datasets import load_classification
from aeon.distances import euclidean_distance


In [2]:
# Define a lista de datasets com alta dimensionalidade a serem usados
high_dim_datasets = [
  'ACSF1',
  'CinCECGTorso',
  'EOGHorizontalSignal',
  'EOGVerticalSignal',
  'EthanolLevel',
  'HandOutlines',
  'Haptics',
  'HouseTwenty',
  'InlineSkate',
  'Mallat',
  'MixedShapesRegularTrain',
  'MixedShapesSmallTrain',
  'Phoneme',
  'PigAirwayPressure',
  'PigArtPressure',
  'PigCVP',
  'Rock',
  'SemgHandGenderCh2',
  'SemgHandMovementCh2',
  'SemgHandSubjectCh2',
  'StarLightCurves',
]

Para facilitar a comparação de séries em diferentes escalas, podemos aplicar uma função de normalização nas séries. Assim todas as séries ficarão com média 0 e desvio padrão 1.

In [3]:
# Para facilitar a comparação de séries em diferentes escalas, 
# podemos aplicar uma função de normalização nas séries.
# Assim todas as séries ficarão com média 0 e desvio padrão 1.
def znorm(x):
  x_znorm = (x - np.mean(x)) / np.std(x)
  return x_znorm

In [4]:
# Define as funções de agregação que podem ser usadas para reduzir a dimensionalidade no PAA
aggregations = {
    'average': lambda x: np.mean(x),
    'median': lambda x: np.median(x),
    'max': lambda x: np.max(x),
    'min': lambda x: np.min(x),
    'sum': lambda x: np.sum(x),
    'variance': lambda x: np.var(x),
    'std': lambda x: np.std(x),
    'iqr': lambda x: np.subtract(*np.percentile(x, [75, 25])),
    'first': lambda x: x[0],
    'central': lambda x: x[len(x)//2],
    'last': lambda x: x[-1],
    'max-min': lambda x: np.max(x) - np.min(x),
    'avg-max': lambda x: np.abs((np.mean(x) - np.max(x))),
    'avg-min': lambda x: np.abs((np.mean(x) - np.min(x))),
    'random': lambda x: np.random.choice(x)
}

In [5]:
# Implementa o algoritmo PAA (Piecewise Aggregate Approximation)
# Recebe uma série temporal 's', o tamanho desejado 'w' e a função de agregação 'agg'
def PAA(s, w, agg='average'):
    if agg not in aggregations:
        raise ValueError(f"Função de agregação '{agg}' é inválida ou não suportada.")

    n = len(s)
    s = np.array(s)

    # Aqui criamos n valores uniformemente espaçados entre 0 e w (exclusivo).
    # Por exemplo: n=6 e w=2 será [0,0,0,1,1,1]
    idx = np.floor(np.linspace(0, w, n, endpoint=False)).astype(int)

    # Aqui iteramos sobre o tamanho desejado 'w' e aplicamos uma máscara para selecionar os pontos da série que pertencem a cada segmento.
    # Por exemplo: n=6 e w=2, o idx resultante será: 
    #              [True, True, True, False, False, False] para w = 0
    #              Assim podemos selecionar os pontos da série que pertencem a cada segmento.
    res = [aggregations[agg](s[idx == i]) for i in range(w)]
    # Normalizamos a série reduzida utilizando Z-Norm
    res = znorm(res)

    return np.array(res)

In [6]:
# Define as taxas de redução de dimensionalidade a serem testadas
reduction_rates = [
    0.9,
    0.8,
    0.7,
    0.6,
    0.5,
    0.4,
    0.3,
    0.2,
    0.1
]

In [30]:
# Importa a função para calcular distâncias de forma eficiente
from scipy.spatial.distance import pdist, squareform

# Função para calcular a matriz de distância entre todas as séries temporais em um conjunto de dados X
def get_distance_matrix(X):
    # 1. Cria uma cópia de X para evitar modificar o original
    X_cp = np.array(X)

    # 2. Remove dimensões unitárias para garantir que X seja 2D (N x M)
    # Por se tratar de séries temporais univariadas, o formato original é 
    # (N x 1 x M) e sera convertido para (N x M)
    X_2d = X_cp.squeeze()

    # 3. pdist calcula as N*(N-1)/2 distâncias únicas
    condensed_dist = pdist(X_2d, metric='euclidean')

    # 4. squareform monta a matriz N x N simétrica completa
    distance_matrix = squareform(condensed_dist)
    
    return distance_matrix

In [22]:
# Função para obter os K vizinhos mais proximos de uma serie (s) por meio de uma matriz de distâncias
def get_k_nearest_neighbors(dist_matrix, point_idx, k):    
    # 1. Pega a linha de distâncias para o nosso ponto
    distances_from_point = dist_matrix[point_idx]
    
    # 2. Obtém os índices ordenados pela distância
    #    Ex: sorted_indices[0] será o próprio point_idx (dist 0)
    sorted_indices = np.argsort(distances_from_point)
    
    # 3. Pega os K vizinhos mais próximos.
    #    Pulamos o primeiro índice (posição 0), pois é o próprio ponto.
    #    Pegamos da posição 1 (o vizinho mais próximo) até k+1.
    k_nearest_indices = sorted_indices[1 : k + 1]
    
    # 4. Pega as distâncias correspondentes a esses vizinhos
    k_nearest_distances = distances_from_point[k_nearest_indices]
    
    return k_nearest_indices, k_nearest_distances

In [32]:
d = 'HouseTwenty'
X, y = load_classification(d, split=None)

print(f'Dataset: {d}, Shape original: {X.shape}')

print("Normalizando séries com sua função znorm...")
X_norm = np.array([[znorm(s[0])] for s in X])

print("Calculando a matriz de distância...")
dist_matrix = get_distance_matrix(X_norm)

print(f"Shape final: {dist_matrix.shape}")

print("\nMatriz de Distância (Início):")
print(dist_matrix[:5, :5])

#Exibe os k vizinhos mais próximos de uma série específica
series_index = 5  # Índice da série que queremos analisar
k = 3

nearest_indices, nearest_distances = get_k_nearest_neighbors(dist_matrix, series_index, k)
print(f'Séries mais próximas da série {series_index}: {nearest_indices} com distâncias {nearest_distances}')

Dataset: HouseTwenty, Shape original: (159, 1, 2000)
Normalizando séries com sua função znorm...
Calculando a matriz de distância...
Shape final: (159, 159)

Matriz de Distância (Início):
[[ 0.         56.13328025 54.34043454 54.15500871 67.07087115]
 [56.13328025  0.         61.12841113 62.47950359 63.62144155]
 [54.34043454 61.12841113  0.         46.69631666 66.82907255]
 [54.15500871 62.47950359 46.69631666  0.         67.17988739]
 [67.07087115 63.62144155 66.82907255 67.17988739  0.        ]]
Séries mais próximas da série 5: [139 107   3] com distâncias [44.33162092 46.05375307 47.36015159]


In [34]:
# Reduz a dimensionalidade das séries utilizando PAA e calcula a nova matriz de distâncias
d = 'HouseTwenty'
X, y = load_classification(d, split=None)

agg = 'median'
reduction_rate = 0.5
w = int(len(X[0][0]) * (1 - reduction_rate))

print(f'Reduzindo para w={w} pontos por série utilizando PAA com agregação {agg}.')
X_reduced = np.array([[PAA(s[0], w, agg=agg)] for s in X])

# Inicializa a nova matriz de distâncias
dist_matrix_reduced = get_distance_matrix(X_reduced)

# Exibe a nova matriz de distâncias
print(dist_matrix_reduced.shape)

# Exibe as k=3 series mais próximas de uma series qualquer do dataset reduzido
series_index = 5  # Índice da série que queremos analisar
k = 3

nearest_indices, nearest_distances = get_k_nearest_neighbors(dist_matrix_reduced, series_index, k)
print(f'Séries mais próximas da série {series_index}: {nearest_indices} com distâncias {nearest_distances}')

Reduzindo para w=1000 pontos por série utilizando PAA com agregação median.
(159, 159)
Séries mais próximas da série 5: [139 107   3] com distâncias [29.95126277 32.07611833 33.22822385]
